# MARATONA BEHIND THE CODE 2020

## DESAFIO 6 - LIT

<hr>

## Installing Libs

In [11]:
%%time
!pip install scikit-learn==0.22.0 --upgrade
!pip install scikit-learn --upgrade
!pip install xgboost --upgrade

  Using cached https://files.pythonhosted.org/packages/e9/fc/37c2706fe0d252e89c49f0c94b94b27878f75a372ca7e5e7ea7583f61c79/scikit_learn-0.22-cp36-cp36m-win_amd64.whl
  Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2


<hr>

## Imports

In [4]:
import pandas as pd










### Download training data (csv) to Pandas Dataframe

In [53]:
#!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset-3/master/training_dataset.csv
#df_training_dataset = pd.read_csv(r'training_dataset.csv')
df_training_dataset = pd.read_csv(r'C:/Users/MARCOSPAULOFerreira/Anaconda2/MARATONA2020/desafio-6-2020/dataset/training_dataset.csv')
df_training_dataset.tail()

,id,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,importante_ter_certificado,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,categoria
15852,620397030.0,Bacharelado,UFF,Outros,Borracha,0.0,0.0,1.0,8.0,NaN,10.0,NaN,NaN,NaN,perfil6
15853,229931283.0,Bacharelado,NaN,Advogado,NaN,0.0,0.0,1.0,7.0,Linkedin,42.0,17.0,15.0,NaN,perfil5
15854,480674907.0,Tecnólogo,UNIP,Sócio/Dono/Proprietário,NaN,0.0,NaN,1.0,7.0,Outros,30.0,9.0,8.0,0.0,perfil5
15855,559626807.0,SEM FORMAÇÃO,UNIVERSIDADE NOVE DE JULHO,Advogado,Estado,0.0,0.0,1.0,10.0,NaN,226.0,102.0,93.0,1.0,perfil1
15856,743652801.0,NaN,FGV-RJ,NaN,Siderurgica,1.0,1.0,1.0,9.0,Outros,125.0,98.0,97.0,1.0,perfil1


In [55]:
df_training_dataset.nunique()

id                            13945
graduacao                         6
universidade                     21
profissao                        12
organizacao                      11
pretende_fazer_cursos_lit         2
interesse_mba_lit                 2
importante_ter_certificado        1
horas_semanais_estudo             9
como_conheceu_lit                 9
total_modulos                   578
modulos_iniciados               372
modulos_finalizados             339
certificados                     23
categoria                         6
dtype: int64

In [56]:
# Exibindo os dados ausentes do conjunto de dados.
print("Valores nulos no df_training_dataset antes das transformaçoes: \n\n{}\n".format(df_training_dataset.isnull().sum(axis = 0)))

Valores nulos no df_training_dataset antes das transformaçoes: 

id                            1912
graduacao                     1907
universidade                  1937
profissao                     1880
organizacao                   1896
pretende_fazer_cursos_lit     1868
interesse_mba_lit             1854
importante_ter_certificado    1939
horas_semanais_estudo         1898
como_conheceu_lit             1942
total_modulos                 1870
modulos_iniciados             1813
modulos_finalizados           1933
certificados                  1878
categoria                        0
dtype: int64



## Pre-processando o dataset antes do treinamento

In [57]:
#Nan to Zero nas colunas pretende_fazer_cursos_lit, interesse_mba_lit,
#importante_ter_certificado, modulos_finalizados, certificados
f = ['pretende_fazer_cursos_lit','interesse_mba_lit','importante_ter_certificado','modulos_finalizados','certificados']
values = {f[0]: 0, f[1]: 0, f[2]: 0, f[3]: 0, f[4]: 0}
df_training_dataset.fillna(value=values,inplace=True)

#Average: horas_semanais_estudo
df_training_dataset['horas_semanais_estudo'].fillna((df_training_dataset['horas_semanais_estudo'].mean()), inplace=True)

#modulos_iniciados Se for Nan transformar no mesmo valor de modulos_finalizados
df_training_dataset.modulos_iniciados.fillna(df_training_dataset.modulos_finalizados, inplace=True)

#total_modulos - Se for Nan transformar no mesmo valor de modulos_iniciados.
df_training_dataset.total_modulos.fillna(df_training_dataset.modulos_iniciados, inplace=True)

#Nan to 'Nao Informado' nas colunas graduacao, universidade, profissao, organizacao, como_conheceu_lit
f = ['graduacao','universidade','profissao','organizacao','como_conheceu_lit']
values = {f[0]: 'Nao Informado', f[1]: 'Nao Informado', f[2]: 'Nao Informado', f[3]: 'Nao Informado', f[4]: 'Nao Informado'}
df_training_dataset.fillna(value=values,inplace=True)

# Exibindo os dados ausentes do conjunto de dados apos fillna
print("Valores nulos no df_training_dataset apos transformaçoes: \n\n{}\n".format(df_training_dataset.isnull().sum(axis = 0)))

Valores nulos no df_training_dataset antes da transformação: 

id                            1912
graduacao                        0
universidade                     0
profissao                        0
organizacao                      0
pretende_fazer_cursos_lit        0
interesse_mba_lit                0
importante_ter_certificado       0
horas_semanais_estudo            0
como_conheceu_lit                0
total_modulos                    0
modulos_iniciados                0
modulos_finalizados              0
certificados                     0
categoria                        0
dtype: int64



### Tratamento de de variáveis categóricas
#### Label Encoding

In [59]:
from sklearn.preprocessing import LabelEncoder
df_train = df_training_dataset.copy(deep=True)

In [60]:
#Codificar variaves categoricas com Label Encoding
le_graduacao = LabelEncoder()
le_universidade = LabelEncoder()
le_profissao = LabelEncoder()
le_organizacao = LabelEncoder()
le_como_conheceu_lit = LabelEncoder()

df_train["code_graduacao"] = le_graduacao.fit_transform(df_train["graduacao"])
df_train["code_universidade"] = le_graduacao.fit_transform(df_train["universidade"])
df_train["code_profissao"] = le_graduacao.fit_transform(df_train["profissao"])
df_train["code_organizacao"] = le_graduacao.fit_transform(df_train["organizacao"])
df_train["code_como_conheceu_lit"] = le_graduacao.fit_transform(df_train["como_conheceu_lit"])

df_train.head()

,id,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,importante_ter_certificado,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,categoria,code_graduacao,code_universidade,code_profissao,code_organizacao,code_como_conheceu_lit
0,915132186.0,Especialização,UNIP,Analista Senior,Eletrodomesticos,1.0,1.0,1.0,6.0,Google,28.0,5.0,4.0,0.0,perfil6,1,14,2,1,1
1,183037660.0,MBA,Nao Informado,Supervisor,e-commerce,0.0,0.0,1.0,6.0,Nao Informado,66.0,36.0,34.0,0.0,perfil2,3,6,11,10,6
2,681614019.0,Bacharelado,UNICAMP,Outros,e-commerce,1.0,1.0,1.0,10.0,Saint Paul,27.0,27.0,27.0,0.0,perfil2,0,13,9,10,8
3,398921329.0,SEM FORMAÇÃO,UFRN,Outros,Eletroeletronicos,0.0,1.0,1.0,5.0,Instagram,29.0,10.0,6.0,0.0,perfil5,5,11,9,2,2
4,522842284.0,Tecnólogo,INSPER INSTITUTO DE ENSINO E PESQUISA,SEM EXPERIÊNCIA,Eletrodomesticos,0.0,0.0,1.0,7.0,Google,67.0,49.0,44.0,0.0,perfil2,6,5,10,1,1


## Treinando um classificador XGBOOST

### Selecionando FEATURES e definindo a variável TARGET

In [61]:
df_train.columns

Index(['id', 'graduacao', 'universidade', 'profissao', 'organizacao',
       'pretende_fazer_cursos_lit', 'interesse_mba_lit',
       'importante_ter_certificado', 'horas_semanais_estudo',
       'como_conheceu_lit', 'total_modulos', 'modulos_iniciados',
       'modulos_finalizados', 'certificados', 'categoria', 'code_graduacao',
       'code_universidade', 'code_profissao', 'code_organizacao',
       'code_como_conheceu_lit'],
      dtype='object')

In [67]:
features = [ 'pretende_fazer_cursos_lit', 'interesse_mba_lit',
             'importante_ter_certificado', 'horas_semanais_estudo',
             'total_modulos', 'modulos_iniciados', 'modulos_finalizados',
             'certificados', 'code_graduacao', 'code_universidade',
             'code_profissao', 'code_organizacao', 'code_como_conheceu_lit'
           ]
target = ['categoria']  ## NÃO TROQUE O NOME DA VARIÁVEL TARGET.

In [68]:
# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_train[features]
y = df_train[target]

In [70]:
%%time
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import PassiveAggressiveClassifier

outcome = []
model_names = []
models = [
          ('LR ', LogisticRegression(solver='lbfgs',max_iter=10000)),  
          ('DTC', DecisionTreeClassifier()),       
          ('KNN', KNeighborsClassifier()),         
          #('SVM', SVC(gamma='auto')),              
          ('LDA', LinearDiscriminantAnalysis()),  
          ('GNB', GaussianNB()),                   
          ('RFC', RandomForestClassifier(n_estimators=100)),
          ('XGB', XGBClassifier()),             
          ('PAC', PassiveAggressiveClassifier(max_iter=1000, tol=1e-3))
         ]


for model_name, model in models:
    k_fold_validation = model_selection.KFold(n_splits=10, shuffle=True)
    results = model_selection.cross_val_score(model, X, y.values.ravel(), cv=k_fold_validation, scoring='accuracy')
    outcome.append(results)
    model_names.append(model_name)
    output_message = "%s| Mean=%f STD=%f" % (model_name, results.mean(), results.std())
    print(output_message)



LR | Mean=0.579744 STD=0.016257
DTC| Mean=0.643498 STD=0.013076
KNN| Mean=0.611654 STD=0.007066
LDA| Mean=0.473607 STD=0.011569
GNB| Mean=0.433500 STD=0.015246
RFC| Mean=0.808728 STD=0.005555
XGB| Mean=0.803178 STD=0.007575
PAC| Mean=0.447631 STD=0.095565
Wall time: 5min 3s


#### 1st - RFC
#### 2nd - XGB

In [75]:
#GridSearchCV for Hyperparm tuning

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score, make_scorer, f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import StratifiedKFold

rfc=RandomForestClassifier()
param_grid = { 
    'n_estimators': [200, 500, 700, 1000, 1500, 2000],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [2,3,4,5,6,7,8,10],
    'criterion' :['gini', 'entropy']
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, verbose=1, n_jobs=-1)
CV_rfc.fit(X, y.values.ravel())

# Dictionary of best parameters
best_pars = grid.best_params_
best_score = grid.best_score_
print(best_pars)
print(best_score)


Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 32.8min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 58.8min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 84.1min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 105.5min finished


NameError: name 'grid' is not defined

In [76]:
# Dictionary of best parameters
best_pars = CV_rfc.best_params_
best_score = CV_rfc.best_score_
print(best_pars)
print(best_score)

{'criterion': 'gini', 'max_depth': 10, 'max_features': 'auto', 'n_estimators': 1500}
0.8082864571465158


## Scoring dos dados necessários para entregar a solução

Como entrega da sua solução, esperamos os resultados classificados no seguinte dataset chamado "to_be_scored.csv":

### Download da "folha de respostas"

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset-3/master/to_be_scored.csv
df_to_be_scored = pd.read_csv(r'to_be_scored.csv')
df_to_be_scored.tail()

# Atenção!

O dataframe ``to_be_scored`` é a sua "folha de respostas". Note que a coluna "categoria" não existe nessa amostra, que não pode ser então utilizada para treino de modelos de aprendizado supervisionado.

In [ ]:
df_to_be_scored.info()

<hr>

# Atenção!

# Para poder aplicar seu modelo e classificar a folha de respostas, você precisa primeiro aplicar as mesmas transformações com colunas que você aplicou no dataset de treino.

# Não remova ou adicione linhas na folha de respostas. 

# Não altere a ordem das linhas na folha de respostas.

# Ao final, as 1000 entradas devem estar classificadas, com os valores previstos em uma coluna chamada "target"

<hr>

Na célula abaixo, repetimos rapidamente os mesmos passos de pré-processamento usados no exemplo dado com árvore de decisão

In [ ]:
# 1 - Removendo linhas com valores NaN em "certificados" e "total_modulos"
df_to_be_scored_1 = df_to_be_scored.dropna(axis='index', how='any', subset=['certificados', 'total_modulos'])

# 2 - Inputando zeros nos valores faltantes
impute_zeros.fit(X=df_to_be_scored_1)
df_to_be_scored_2 = pd.DataFrame.from_records(
    data=impute_zeros.transform(
        X=df_to_be_scored_1
    ),
    columns=df_to_be_scored_1.columns
)

# 3 - Remoção de colunas
df_to_be_scored_3 = df_to_be_scored_2.drop(columns=['id', 'graduacao', 'universidade', 'organizacao', 'como_conheceu_lit'], inplace=False)

# 4 - Encoding com "dummy variables"
df_to_be_scored_4 = pd.get_dummies(df_to_be_scored_3, columns=['profissao'])

df_to_be_scored_4.tail()

<hr>

Pode ser verificado abaixo que as colunas da folha de resposta agora são idênticas às que foram usadas para treinar o modelo:

In [ ]:
df_training[
    [
        'pretende_fazer_cursos_lit', 'interesse_mba_lit',
        'importante_ter_certificado', 'horas_semanais_estudo', 'total_modulos',
        'modulos_iniciados', 'modulos_finalizados', 'certificados',
        'profissao_0', 'profissao_Advogado', 'profissao_Analista',
        'profissao_Analista Senior', 'profissao_Assessor',
        'profissao_Coordenador', 'profissao_Diretor', 'profissao_Engenheiro',
        'profissao_Gerente', 'profissao_Outros', 'profissao_SEM EXPERIÊNCIA',
        'profissao_Supervisor', 'profissao_Sócio/Dono/Proprietário'
    ]
].columns

In [ ]:
df_to_be_scored_4.columns

# Atenção

Para todas colunas que não existirem no "df_to_be_scored", você pode usar a técnica abaixo para adicioná-las:

In [ ]:
df_to_be_scored_4['profissao_0'] = 0

In [ ]:
y_pred = dtc.predict(df_to_be_scored_4)
df_to_be_scored_4['target'] = y_pred
df_to_be_scored_4.tail()

### Salvando a folha de respostas como um arquivo .csv para ser submetido

In [ ]:
project.save_data(file_name="results.csv", data=df_to_be_scored_4.to_csv(index=False))

# Atenção

# A execução da célula acima irá criar um novo "data asset" no seu projeto no Watson Studio. Você precisará realizar o download deste arquivo juntamente com este notebook e criar um arquivo zip com os arquivos **results.csv** e **notebook.ipynb** para submissão. (os arquivos devem estar nomeados desta forma)

<hr>

## Parabéns!

Se você já está satisfeito com a sua solução, vá até a página abaixo e envie os arquivos necessários para submissão.

# https://lit.maratona.dev
